In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import io
from multiprocessing import Pool

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

response = requests.get(url)
soup = BeautifulSoup(response.text,"html.parser")

In [3]:
header = soup.find_all('th')
headerList = []
for x in header:
    headerList.append(x.get_text(strip=True))
headerList = headerList[0:9]

d = soup.find('table')

data = []

In [4]:
for row in d.find_all('tr'):
    sampleList = []
    for td in row.find_all('td'):
        sampleList.append(td.get_text(strip=True))
    data.append(sampleList)

In [5]:
data.pop(0)

df = pd.DataFrame(data, columns = headerList)

In [6]:
df

,Symbol,Security,SEC filings,GICSSector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",1976-08-09,0000066740,1902
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,0000001800,1888
2,ABBV,AbbVie Inc.,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,0001551152,2013 (1888)
3,ABMD,ABIOMED Inc,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,0000815094,1981
4,ACN,Accenture plc,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,0001467373,1989
...,...,...,...,...,...,...,...,...,...
500,YUM,Yum! Brands Inc,reports,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,0001041061,1997
501,ZBRA,Zebra Technologies,reports,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,0000877212,1969
502,ZBH,Zimmer Biomet,reports,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,0001136869,1927
503,ZION,Zions Bancorp,reports,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,0000109380,1873


In [7]:
symbol = df['Symbol']
period1 = "1483228800"
period2 = "1609545600"
symbol = symbol.head()

In [8]:
def scrape(l):
    link = "https://query1.finance.yahoo.com/v7/finance/download/"+l+"?period1="+period1+"&period2="+period2+"&interval=1wk&events=history&includeAdjustedClose=true"
    print(link)
    response = requests.get(link)
    print(response)
    bytes_io = io.BytesIO(response.content)
    print(bytes_io)
    dataframe = pd.read_csv(bytes_io, error_bad_lines=False)
    dataframe['Symbol'] = l
    return dataframe

In [9]:
# p = Pool(30)
# df_list = p.map(scrape,symbol)
for s in symbol:
    scrape(symbol)
p.close()
for t in df_list:
    if len(t.columns) > 3:
        close = t['Close'].tolist()
        gl = []
        for i in range(1,len(close)):
            gl.append(((close[i]-close[i-1])/close[i-1])*100)
        gl.insert(0,((gl[0]+gl[1]+gl[2])/3)*100)
        t['Gain or Loss'] = gl

table = pd.concat(df_list).reset_index(drop=True)

table.to_csv('Weekly.csv')

0    https://query1.finance.yahoo.com/v7/finance/do...
1    https://query1.finance.yahoo.com/v7/finance/do...
2    https://query1.finance.yahoo.com/v7/finance/do...
3    https://query1.finance.yahoo.com/v7/finance/do...
4    https://query1.finance.yahoo.com/v7/finance/do...
Name: Symbol, dtype: object


InvalidSchema: No connection adapters were found for '0    https://query1.finance.yahoo.com/v7/finance/do...\n1    https://query1.finance.yahoo.com/v7/finance/do...\n2    https://query1.finance.yahoo.com/v7/finance/do...\n3    https://query1.finance.yahoo.com/v7/finance/do...\n4    https://query1.finance.yahoo.com/v7/finance/do...\nName: Symbol, dtype: object'